In [1]:
# MISC
import os
import sys

sys.path.append("..")
os.environ['MUJOCO_GL']='egl'
os.environ['CUDA_VISIBLE_DEVICES']='0'

# import shutup
# shutup.please()

import jax
import jax.numpy as jnp

import numpy as np
from functools import partial

# VIS
import matplotlib
import matplotlib.pyplot as plt
import mediapy
from rich.pretty import pprint
from tqdm.auto import tqdm

from hydra import initialize, compose
from omegaconf import OmegaConf

plt.style.use(['seaborn-v0_8-colorblind', 'seaborn-v0_8-notebook'])
colors = plt.rcParams['axes.prop_cycle'].by_key()['color'] 

GLOBAL_KEY = jax.random.key(42)

In [ ]:
from envs.custom_mazes.darkroom import FourRoomsMazeEnv, Maze

test = FourRoomsMazeEnv(Maze(seed=42, maze_type='fourrooms_random_layouts'))
test.reset()
test.render(return_img=False) 

In [ ]:
from envs.minigrid.env_utils import random_exploration_fourrooms, q_learning_fourrooms

train_layout_data = []

NUM_TRAIN_LAYOUTS = 30
# NUM_TRAIN_STEPS = 100
# NUM_TRAIN_EPISODES = 100

seeds = np.arange(0, NUM_TRAIN_LAYOUTS)

# for i in tqdm(range(NUM_TRAIN_LAYOUTS)):
#     env = FourRoomsMazeEnv(Maze(seed=seeds[i], maze_type='fourrooms_random_layouts'), max_steps=NUM_TRAIN_STEPS)
#     dataset, env = q_learning_fourrooms(env, num_episodes=NUM_TRAIN_EPISODES, layout_type=i, epsilon=0.8, num_mdp=NUM_TRAIN_LAYOUTS)
#     train_layout_data.append(dataset)

NUM_TRAIN_STEPS = 99
NUM_TRAIN_EPISODES = 1000

for i in tqdm(range(NUM_TRAIN_LAYOUTS)):
    env = FourRoomsMazeEnv(Maze(seed=seeds[i], maze_type='fourrooms_random_layouts'), max_steps=NUM_TRAIN_STEPS)
    dataset, env = random_exploration_fourrooms(env, num_episodes=NUM_TRAIN_EPISODES, layout_type=i, num_mdp=NUM_TRAIN_LAYOUTS)
    train_layout_data.append(dataset)
    
pprint(jax.tree.map(lambda x: x.shape, train_layout_data[0]))

coverage_map = np.zeros(shape=env.maze.size)
for layout in train_layout_data:
    for obs in layout['observations']:
        obs = obs.astype(np.int16)
        coverage_map[obs[1], obs[0]] += 1
        
plt.imshow(coverage_map, cmap='inferno', vmin=0)
plt.colorbar()
plt.show()

In [ ]:
import functools
from utils.datasets import Dataset, GCDataset

with initialize(version_base=None, config_path="../configs/"):
    fb_config = compose(config_name='entry.yaml', overrides=['experiment=fb_dynamics_discrete_4rooms.yaml',
                                                            f'agent.number_of_meta_envs={NUM_TRAIN_LAYOUTS}',
                                                            f'+agent.kappa=150',
                                                            f'agent.z_dim=150',
                                                            f'agent.output_dim=150',
                                                            f'agent.z_mix_ratio=0.2'])
    fb_config = OmegaConf.to_container(fb_config, resolve=True)
    pprint(fb_config)

def concatenate_dicts(dict1, dict2):
    return jax.tree.map(lambda x, y: jnp.concatenate([x, y]), dict1, dict2)

whole_data = functools.reduce(concatenate_dicts, train_layout_data)
# SAVING
# np.save(f"../aux_data/fourrooms_meta{NUM_TRAIN_LAYOUTS}_data", arr=jax.device_get(whole_data))
# whole_data = np.load(f"/home/m_bobrin/ZeroShotRL/aux_data/fourrooms_meta{NUM_TRAIN_LAYOUTS}_data.npy", allow_pickle=True).item()

print(jax.tree.map(lambda x: x.shape, whole_data))
whole_dataset = Dataset.create(**jax.device_get(whole_data))
gc_whole_dataset = GCDataset(whole_dataset, config=fb_config['agent'])

In [ ]:
sys.path.append("..")
from agents.test_fb import ForwardBackwardAgent

gc_whole_dataset = GCDataset(whole_dataset, config=fb_config['agent'])
example_batch = gc_whole_dataset.sample(1)
fb_agent = ForwardBackwardAgent.create(
    0,
    example_batch['observations'],
    np.full_like(example_batch['actions'], env.action_space.n - 1),
    config=fb_config['agent']
)
batch = gc_whole_dataset.sample(2, layout_type=None, get_traj_batch=True, context_length=NUM_TRAIN_STEPS)[1]
# with jax.disable_jit():
fb_agent, info = fb_agent.update(batch, train_context_embedding=False)

In [ ]:
from envs.custom_mazes.env_utils import policy_image_fourrooms, value_image_fourrooms
from functools import partial
from utils.evaluation import supply_rng
from IPython.display import clear_output

from envs.env_utils import EpisodeMonitor
from utils.evaluation import evaluate_fourrooms_dynamics
from collections import defaultdict


def visualize_value_image(env, layout_type, task_num):
    env.reset()
    observation, info = env.setup_goals(seed=None, task_num=task_num)
    goal = info.get("goal_pos", None)
    mdp_type=None
    if fb_config['agent']['use_context']:
        dataset_inference, env = random_exploration_fourrooms(env, num_episodes=1, layout_type=layout_type, num_mdp=NUM_TRAIN_LAYOUTS)
        print(jax.tree.map(lambda x: x.shape, dataset_inference))
        dynamics_embedding_mean, dynamics_mean_std = fb_agent.network.select('dynamic_transformer')(dataset_inference['observations'][None], dataset_inference['actions'][None, :, None],
                                                                                    dataset_inference['next_observations'][None], train=False, return_embedding=True)
        dynamics_embedding = dynamics_embedding_mean + jax.random.normal(key=GLOBAL_KEY, shape=dynamics_embedding_mean.shape) * jnp.exp(dynamics_mean_std)
        dynamics_embedding = dynamics_embedding.squeeze()

    latent_z = jax.device_get(fb_agent.infer_z(goal, mdp_num=None, dynamics_embedding=dynamics_embedding)[None])
    N, M = env.maze.size
    pred_value_img = value_image_fourrooms(env, example_batch,N=N, M=M,
                                value_fn=partial(fb_agent.predict_q, z=latent_z, mdp_num=mdp_type[None] if mdp_type is not None else None,
                                                dynamics_embedding=dynamics_embedding[None]),
                                action_fn=None, goal=goal)
    return pred_value_img

def visualize_policy(env, layout_type, task_num):
    env.reset()
    observation, info = env.setup_goals(seed=None, task_num=task_num)
    goal = info.get("goal_pos", None)
    if fb_config['agent']['use_context']:
        dataset_inference, env = random_exploration_fourrooms(env, num_episodes=1, layout_type=layout_type, num_mdp=NUM_TRAIN_LAYOUTS)
        print(jax.tree.map(lambda x: x.shape, dataset_inference))
        dynamics_embedding_mean, dynamics_mean_std = fb_agent.network.select('dynamic_transformer')(dataset_inference['observations'][None], dataset_inference['actions'][None,:,None],
                                                                                    dataset_inference['next_observations'][None], train=False, return_embedding=True)
        dynamics_embedding = dynamics_embedding_mean + jax.random.normal(key=GLOBAL_KEY, shape=dynamics_embedding_mean.shape) * jnp.exp(dynamics_mean_std)
        dynamics_embedding=dynamics_embedding.squeeze()
        mdp_type=None
        
    latent_z = fb_agent.infer_z(goal, mdp_num=mdp_type, dynamics_embedding=dynamics_embedding)
    start = info.get("start_pos", None)
    example_batch = whole_dataset.sample(1)
    mdp_type=None
    N, M = env.maze.size
    pred_policy_img = policy_image_fourrooms(env, example_batch, N=N, M=M,
                                                    action_fn=partial(supply_rng(fb_agent.sample_actions,
                                                                                rng=jax.random.PRNGKey(np.random.randint(0, 2**32))), latent_z=latent_z,
                                                                    mdp_num=None, dynamics_embedding=dynamics_embedding[None], temperature=0.0),
                                                    goal=goal)
    return pred_policy_img

pbar = tqdm(range(150_000))
eval_history_train = []
eval_history_test = []

for update_step in pbar:
    batch = gc_whole_dataset.sample(fb_config['agent']['batch_size'], layout_type=None, context_length=NUM_TRAIN_STEPS, get_traj_batch=True)[1]
    fb_agent, info = fb_agent.update(batch, train_context_embedding=True if update_step < 60_000 else False)
    
    if update_step % 20_000 == 0:
        clear_output()
        env = FourRoomsMazeEnv(Maze(seed=4, maze_type='fourrooms_random_layouts'), max_steps=NUM_TRAIN_STEPS)
        fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(20, 10))
        pred_policy_img = visualize_policy(env, layout_type=0, task_num=0)
        
        ax[0, 0].imshow(pred_policy_img)
        pred_value_img = visualize_value_image(env, layout_type=0, task_num=0)
        ax[0, 1].imshow(pred_value_img)
        
        env = FourRoomsMazeEnv(Maze(seed=3, maze_type='fourrooms_random_layouts'), max_steps=NUM_TRAIN_STEPS)
        pred_policy_img = visualize_policy(env, layout_type=1, task_num=2)
        ax[1, 0].imshow(pred_policy_img)
        pred_value_img = visualize_value_image(env, layout_type=1, task_num=2)
        ax[1, 1].imshow(pred_value_img)
        
        fig.suptitle(f"Training step: {update_step}")   
        plt.tight_layout()
        display(fig)
        plt.close(fig)
    
    if (update_step > 60_000 and update_step % 10_000 == 0) or update_step == 60_000:
        eval_metrics = {}
        overall_metrics = defaultdict(list)

        for task_id in range(4): # static for 4 rooms
            for env_id in range(fb_config['agent']['number_of_meta_envs']):
                env = FourRoomsMazeEnv(Maze(seed=env_id, maze_type='fourrooms_random_layouts'), max_steps=fb_config['agent']['context_len'])
                env = EpisodeMonitor(env, filter_regexes=['.*privileged.*', '.*proprio.*'])
                env.reset(options={"start": (1, 1)})
                dataset_inference, env = random_exploration_fourrooms(env, num_episodes=1, layout_type=0, num_mdp=fb_config['agent']['number_of_meta_envs'])
                dynamics_embedding_mean, dynamics_mean_std = fb_agent.network.select('dynamic_transformer')(dataset_inference['observations'][None], dataset_inference['actions'][None,:,None],
                                                                                            dataset_inference['next_observations'][None], train=False, return_embedding=True)
                dynamics_embedding = dynamics_embedding_mean + jax.random.normal(key=GLOBAL_KEY, shape=dynamics_embedding_mean.shape) * jnp.exp(dynamics_mean_std)
                dynamics_embedding=dynamics_embedding.squeeze()
                eval_info, _, _ = evaluate_fourrooms_dynamics(
                        agent=fb_agent,
                        dynamics_embedding=dynamics_embedding,
                        env=env,
                        task_id=task_id,
                        config=None,
                        num_eval_episodes=10,
                        num_video_episodes=0,
                        video_frame_skip=1,
                        eval_temperature=0.0,
                        eval_gaussian=None
                    )
                eval_metrics.update(
                    {f'evaluation/task_{task_id}_{k}': v for k, v in eval_info.items() if k != 'total.timesteps'}
                )
                for k, v in eval_info.items():
                    overall_metrics[k].append(v)
                    
        for k, v in overall_metrics.items():
            eval_metrics[f'evaluation/overall_{k}_train'] = np.mean(v)
            
        eval_history_train.append(eval_metrics['evaluation/overall_episode.final_reward_train'])
        
        eval_metrics = {}
        overall_metrics = defaultdict(list)
                    
        for task_id in range(4):
            for env_id in range(NUM_TRAIN_LAYOUTS+40, NUM_TRAIN_LAYOUTS + 60):
                env = FourRoomsMazeEnv(Maze(seed=env_id, maze_type='fourrooms_random_layouts'), max_steps=fb_config['agent']['context_len'])
                env = EpisodeMonitor(env, filter_regexes=['.*privileged.*', '.*proprio.*'])
                env.reset(options={"start": (1, 1)})
                dataset_inference, env = random_exploration_fourrooms(env, num_episodes=1, layout_type=0, num_mdp=NUM_TRAIN_LAYOUTS)
                dynamics_embedding_mean, dynamics_mean_std = fb_agent.network.select('dynamic_transformer')(dataset_inference['observations'][None], dataset_inference['actions'][None,:,None],
                                                                                            dataset_inference['next_observations'][None], train=False, return_embedding=True)
                dynamics_embedding = dynamics_embedding_mean + jax.random.normal(key=GLOBAL_KEY, shape=dynamics_embedding_mean.shape) * jnp.exp(dynamics_mean_std)
                dynamics_embedding=dynamics_embedding.squeeze()
                eval_info, _, _ = evaluate_fourrooms_dynamics(
                        agent=fb_agent,
                        dynamics_embedding=dynamics_embedding,
                        env=env,
                        task_id=task_id,
                        config=None,
                        num_eval_episodes=10, ##
                        num_video_episodes=0,
                        video_frame_skip=1,
                        eval_temperature=0.0,
                        eval_gaussian=None
                    )
                eval_metrics.update(
                    {f'evaluation/task_{task_id}_{k}': v for k, v in eval_info.items() if k != 'total.timesteps'}
                    )
                for k, v in eval_info.items():
                    overall_metrics[k].append(v)
                    
        for k, v in overall_metrics.items():
            eval_metrics[f'evaluation/overall_{k}_ood'] = np.mean(v)
        eval_history_test.append(eval_metrics['evaluation/overall_episode.final_reward_ood'])

In [ ]:
plt.plot(eval_history_test, label='test')
plt.plot(eval_history_train, label='train')
plt.legend()

In [ ]:
plt.plot(eval_history_test, label='test')
plt.plot(eval_history_train, label='train')
plt.legend()

# ZeroShot

In [ ]:
from envs.env_utils import EpisodeMonitor
from utils.evaluation import evaluate_fourrooms_dynamics
from collections import defaultdict

eval_metrics = {}
overall_metrics = defaultdict(list)

for task_id in range(4): # static for 4 rooms
    for env_id in range(fb_config['agent']['number_of_meta_envs']):
        env = FourRoomsMazeEnv(Maze(seed=env_id, maze_type='fourrooms_random_layouts'), max_steps=fb_config['agent']['context_len'])
        env = EpisodeMonitor(env, filter_regexes=['.*privileged.*', '.*proprio.*'])
        env.reset(options={"start": (1, 1)})
        dataset_inference, env = random_exploration_fourrooms(env, num_episodes=1, layout_type=0, num_mdp=fb_config['agent']['number_of_meta_envs'])
        dynamics_embedding_mean, dynamics_mean_std = fb_agent.network.select('dynamic_transformer')(dataset_inference['observations'][None], dataset_inference['actions'][None,:,None],
                                                                                    dataset_inference['next_observations'][None], train=False, return_embedding=True)
        dynamics_embedding = dynamics_embedding_mean + jax.random.normal(key=GLOBAL_KEY, shape=dynamics_embedding_mean.shape) * jnp.exp(dynamics_mean_std)
        dynamics_embedding=dynamics_embedding.squeeze()
        eval_info, _, _ = evaluate_fourrooms_dynamics(
                agent=fb_agent,
                dynamics_embedding=dynamics_embedding,
                env=env,
                task_id=task_id,
                config=None,
                num_eval_episodes=10,
                num_video_episodes=0,
                video_frame_skip=1,
                eval_temperature=0.0,
                eval_gaussian=None
            )
        eval_metrics.update(
            {f'evaluation/task_{task_id}_{k}': v for k, v in eval_info.items() if k != 'total.timesteps'}
        )
        for k, v in eval_info.items():
            overall_metrics[k].append(v)
            
for k, v in overall_metrics.items():
    eval_metrics[f'evaluation/overall_{k}_train'] = np.mean(v)

In [ ]:
eval_metrics['evaluation/overall_episode.final_reward_train']

In [ ]:
eval_metrics = {}
overall_metrics = defaultdict(list)
            
for task_id in range(4):
    for env_id in range(NUM_TRAIN_LAYOUTS+40, NUM_TRAIN_LAYOUTS + 60):
        env = FourRoomsMazeEnv(Maze(seed=env_id, maze_type='fourrooms_random_layouts'), max_steps=fb_config['agent']['context_len'])
        env = EpisodeMonitor(env, filter_regexes=['.*privileged.*', '.*proprio.*'])
        env.reset(options={"start": (1, 1)})
        dataset_inference, env = random_exploration_fourrooms(env, num_episodes=1, layout_type=0, num_mdp=NUM_TRAIN_LAYOUTS)
        dynamics_embedding_mean, dynamics_mean_std = fb_agent.network.select('dynamic_transformer')(dataset_inference['observations'][None], dataset_inference['actions'][None,:,None],
                                                                                    dataset_inference['next_observations'][None], train=False, return_embedding=True)
        dynamics_embedding = dynamics_embedding_mean + jax.random.normal(key=GLOBAL_KEY, shape=dynamics_embedding_mean.shape) * jnp.exp(dynamics_mean_std)
        dynamics_embedding=dynamics_embedding.squeeze()
        eval_info, _, _ = evaluate_fourrooms_dynamics(
                agent=fb_agent,
                dynamics_embedding=dynamics_embedding,
                env=env,
                task_id=task_id,
                config=None,
                num_eval_episodes=10, ##
                num_video_episodes=0,
                video_frame_skip=1,
                eval_temperature=0.0,
                eval_gaussian=None
            )
        eval_metrics.update(
            {f'evaluation/task_{task_id}_{k}': v for k, v in eval_info.items() if k != 'total.timesteps'}
            )
        for k, v in eval_info.items():
            overall_metrics[k].append(v)
            
for k, v in overall_metrics.items():
    eval_metrics[f'evaluation/overall_{k}_ood'] = np.mean(v)

In [ ]:
eval_metrics['evaluation/overall_episode.final_reward_ood']

In [ ]:
env = FourRoomsMazeEnv(Maze(seed=300, maze_type='fourrooms_random_layouts'), max_steps=NUM_TRAIN_STEPS)
pred_policy_img = visualize_policy(env, layout_type=1, task_num=0)
plt.imshow(pred_policy_img)